In [ ]:
import pandas as pd

import sys
import os
sys.path.append(os.path.abspath('../code'))
import config

languages = ["pap", "en", "nl"]
# d = {k: [] for k in ["passage-id"] + languages}
d = {}

dir = config.jwScrapeTarget["outbblpre"]

for lan in languages:
    books_path = os.path.join(dir, lan)
    for file in os.listdir(books_path)[:]:
        book_path = os.path.join(books_path, file)
        print(book_path)
        
        with open(book_path, encoding="UTF-8") as fileobject:
            current_passage_id = ""
            reading_passage = False
            passage = ""
            for line in fileobject:
                # print(line)
                if "--chapter: " in line: 
                    continue
                elif "--passage_id: " in line: 
                    passage_id = line[:-3].split(": ")[1][1:]
                    # print(passage_id)
                    
                    passage = " ".join(passage.split())
                    passage = passage.replace("*", "")
                    passage = passage.replace("·", "")
                    passage = passage.replace("+", "")

                    if current_passage_id not in d:
                        if lan == "pap" and current_passage_id:
                            d[current_passage_id] = [passage]
                    else:
                        d[current_passage_id].append(passage)
                    
                    current_passage_id = passage_id
                    reading_passage = False
                    passage = ""
                elif "--passage--" in line:
                    reading_passage = True
                elif reading_passage:
                    passage += line
# d

In [ ]:
df = pd.DataFrame.from_dict(d, orient="index")
df = df.reset_index()
df.columns = ["passage-id"] + languages
df = df.sort_values(by="passage-id", ascending=True)

df.to_csv("../data/jw/pap-en-nl.csv")
df